# MCP Registry Demo - Part 4: Manage and Test

This notebook demonstrates how to manage and test the MCP Registry.

## What You'll Learn

- How to add tools to the registry
- How to list and retrieve tools
- How to test registry operations
- How to monitor registry health

## Step 1: Import Libraries and Setup

In [ ]:
import subprocess
import json
import os
import requests
from pathlib import Path
import warnings

# Disable SSL warnings for demo purposes
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

def run_command(cmd, show_output=True):
    """Helper function to run shell commands"""
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if show_output:
        if result.stdout:
            print(result.stdout)
        if result.stderr and result.returncode != 0:
            print(f"Error: {result.stderr}")
    return result.returncode, result.stdout, result.stderr

# Get repository root
repo_root = Path.cwd().parent
config_dir = repo_root / "config"
namespace = "mcp-registry"

print(f"Repository root: {repo_root}")
print(f"Config directory: {config_dir}")
print(f"Target namespace: {namespace}")

## Step 2: Get Registry URL

In [ ]:
print("Getting Registry URL...\n")

returncode, stdout, stderr = run_command(
    f"oc get route mcp-registry -n {namespace} -o json",
    show_output=False
)

if returncode == 0:
    try:
        route = json.loads(stdout)
        host = route['spec']['host']
        tls = route['spec'].get('tls', {})
        protocol = 'https' if tls else 'http'
        registry_url = f"{protocol}://{host}"
        
        print(f"✓ MCP Registry URL: {registry_url}")
        api_base = f"{registry_url}/api/v1"
        print(f"✓ API Base URL: {api_base}")
        
    except (json.JSONDecodeError, KeyError) as e:
        print(f"✗ Error parsing route: {e}")
        registry_url = None
        api_base = None
else:
    print("✗ Could not retrieve route information")
    registry_url = None
    api_base = None

## Step 3: Load Sample Tools Configuration

In [ ]:
print("Loading sample tools configuration...\n")

sample_tools_file = config_dir / "sample-tools.json"

try:
    with open(sample_tools_file, 'r') as f:
        sample_tools = json.load(f)
    
    print(f"✓ Loaded {len(sample_tools['tools'])} sample tools:")
    for tool in sample_tools['tools']:
        print(f"  - {tool['name']} (v{tool['version']}): {tool['description']}")
        
except FileNotFoundError:
    print(f"✗ Sample tools file not found: {sample_tools_file}")
    sample_tools = None
except json.JSONDecodeError as e:
    print(f"✗ Error parsing sample tools: {e}")
    sample_tools = None

## Step 4: Test Registry Health

In [ ]:
print("Testing registry health...\n")

if registry_url:
    try:
        # Test health endpoint
        health_url = f"{registry_url}/health"
        response = requests.get(health_url, timeout=10, verify=False)
        
        if response.status_code == 200:
            print(f"✓ Health check passed")
            print(f"  Response: {response.json()}")
        else:
            print(f"✗ Health check failed with status {response.status_code}")
            
        # Test ready endpoint
        ready_url = f"{registry_url}/ready"
        response = requests.get(ready_url, timeout=10, verify=False)
        
        if response.status_code == 200:
            print(f"✓ Readiness check passed")
            print(f"  Response: {response.json()}")
        else:
            print(f"✗ Readiness check failed with status {response.status_code}")
            
    except requests.exceptions.RequestException as e:
        print(f"✗ Could not connect to registry: {e}")
else:
    print("✗ Registry URL not available")

## Step 5: Add Tools to Registry

In [ ]:
print("Adding tools to registry...\n")

if api_base and sample_tools:
    tools_url = f"{api_base}/tools"
    
    for tool in sample_tools['tools']:
        try:
            response = requests.post(
                tools_url,
                json=tool,
                headers={'Content-Type': 'application/json'},
                timeout=10,
                verify=False
            )
            
            if response.status_code in [200, 201]:
                print(f"✓ Added tool: {tool['name']}")
            else:
                print(f"✗ Failed to add tool {tool['name']}: Status {response.status_code}")
                print(f"  Response: {response.text}")
                
        except requests.exceptions.RequestException as e:
            print(f"✗ Error adding tool {tool['name']}: {e}")
else:
    print("✗ Cannot add tools - API base URL or sample tools not available")

## Step 6: List All Tools

In [ ]:
print("Listing all tools in registry...\n")

if api_base:
    try:
        tools_url = f"{api_base}/tools"
        response = requests.get(tools_url, timeout=10, verify=False)
        
        if response.status_code == 200:
            data = response.json()
            tools = data.get('tools', [])
            
            print(f"✓ Found {len(tools)} tools in registry:\n")
            
            for i, tool in enumerate(tools, 1):
                print(f"{i}. {tool['name']} (v{tool['version']})")
                print(f"   Description: {tool['description']}")
                print(f"   Capabilities: {', '.join(tool.get('capabilities', []))}")
                print()
        else:
            print(f"✗ Failed to list tools: Status {response.status_code}")
            
    except requests.exceptions.RequestException as e:
        print(f"✗ Error listing tools: {e}")
else:
    print("✗ API base URL not available")

## Step 7: Retrieve Specific Tool

In [ ]:
print("Retrieving specific tool details...\n")

tool_name = "filesystem-tool"  # Change this to test different tools

if api_base:
    try:
        tool_url = f"{api_base}/tools/{tool_name}"
        response = requests.get(tool_url, timeout=10, verify=False)
        
        if response.status_code == 200:
            tool = response.json()
            print(f"✓ Tool Details for '{tool_name}':\n")
            print(json.dumps(tool, indent=2))
        elif response.status_code == 404:
            print(f"✗ Tool '{tool_name}' not found in registry")
        else:
            print(f"✗ Failed to retrieve tool: Status {response.status_code}")
            
    except requests.exceptions.RequestException as e:
        print(f"✗ Error retrieving tool: {e}")
else:
    print("✗ API base URL not available")

## Step 8: Test Custom Tool Addition

In [ ]:
print("Adding a custom tool...\n")

custom_tool = {
    "name": "custom-demo-tool",
    "version": "1.0.0",
    "description": "Custom tool created during demo",
    "type": "mcp",
    "capabilities": [
        "demo_capability_1",
        "demo_capability_2"
    ],
    "configuration": {
        "setting1": "value1",
        "setting2": "value2"
    },
    "metadata": {
        "author": "Demo User",
        "license": "Apache-2.0",
        "created": "2024-01-01"
    }
}

if api_base:
    try:
        tools_url = f"{api_base}/tools"
        response = requests.post(
            tools_url,
            json=custom_tool,
            headers={'Content-Type': 'application/json'},
            timeout=10,
            verify=False
        )
        
        if response.status_code in [200, 201]:
            print(f"✓ Custom tool added successfully")
            print(f"\nResponse: {json.dumps(response.json(), indent=2)}")
        else:
            print(f"✗ Failed to add custom tool: Status {response.status_code}")
            print(f"  Response: {response.text}")
            
    except requests.exceptions.RequestException as e:
        print(f"✗ Error adding custom tool: {e}")
else:
    print("✗ API base URL not available")

## Step 9: Monitor Registry Resources

In [ ]:
print("Monitoring registry resources...\n")

# Check pod resource usage
print("Pod Resource Usage:")
run_command(f"oc top pod -n {namespace} -l app=mcp-registry")

print("\nPod Status:")
run_command(f"oc get pods -n {namespace} -l app=mcp-registry -o wide")

print("\nPersistent Volume Claim Usage:")
run_command(f"oc get pvc -n {namespace}")

## Step 10: View Recent Logs

In [ ]:
print("Fetching recent registry logs...\n")

run_command(f"oc logs -n {namespace} deployment/mcp-registry --tail=50")

## Summary

In this notebook, you:
- Tested registry health and readiness
- Added sample tools to the registry
- Listed and retrieved tools from the registry
- Created a custom tool
- Monitored registry resources

## Cleanup (Optional)

To remove the entire demo deployment, run:

```bash
oc delete namespace mcp-registry
oc delete clusterrole toolhive-operator
oc delete clusterrolebinding toolhive-operator
```

## Next Steps

- Explore the [API Reference documentation](../docs/api-reference.md)
- Review [Architecture Overview](../docs/architecture.md)
- Check [Troubleshooting Guide](../docs/troubleshooting.md) for common issues
- Integrate the registry with your MCP clients

## Additional Resources

- [Model Context Protocol Documentation](https://modelcontextprotocol.io/)
- [OpenShift Documentation](https://docs.openshift.com/)
- [Stacklok ToolHive](https://github.com/stacklok/minder)